In [5]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K

In [32]:
def build_CNN_net(width, height, channels, n_labels):
    input_shape = (width, height, channels)
    channel_dim = -1
    
    if K.image_data_format() == 'channels_first':
        input_shape = (channels, width, height)
        channel_dim = 1
    
    ## Build the CNN
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=input_shape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))
    
    # first (and only) set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    # softmax classifier
    model.add(Dense(n_labels))
    model.add(Activation("softmax"))
    
    # return the constructed network architecture
    return model

In [8]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [10]:
# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True, help="path to input dataset (i.e., directory of images)")
ap.add_argument("-m", "--model", required=True, help="path to output model")
ap.add_argument("-l", "--labelbin", required=True, help="path to output label binarizer")
ap.add_argument("-p", "--plot", type=str, default="plot.png", help="path to output accuracy/loss plot")
#args = vars(ap.parse_args())

_StoreAction(option_strings=['-p', '--plot'], dest='plot', nargs=None, const=None, default='plot.png', type=<class 'str'>, choices=None, help='path to output accuracy/loss plot', metavar=None)

## Initialize Variabels


In [54]:
# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
N_EPOCHS = 100
INIT_LR = 1e-3
N_BATCHES = 32
IMAGE_DIMS = (96, 96, 3)
#IMAGE_PATH = if not args: ["dataset"] else '/Users/rafaelferreirarodrigues/Downloads/Notas'
IMAGE_PATH = '/Users/rafaelferreirarodrigues/Downloads/Notas'
MODEL_PATH = './'
PLOT_PATH = './plot.jpg'
SEED = 42

# initialize the data and labels
data = []
labels = []

# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images(IMAGE_PATH)))
random.seed(SEED)
random.shuffle(imagePaths)

[INFO] loading images...


# Load Images

Load all the images in the format: {image path}/{Class}/{image}.jpg

In [14]:
# loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)
    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

In [15]:
print(labels[0:5])

['100 nova', '100 antiga', '200 ', '200 ', '50 nova']


In [16]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(
	data.nbytes / (1024 * 1000.0)))
# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.2, random_state=42)

[INFO] data matrix: 137.16MB


# Incrase the quantity of samples
Increase the number of images because each class has less than 250 samples.

In [17]:
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

In [35]:
# initialize the model
print("[INFO] compiling model...")
model = build_CNN_net(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0], channels=IMAGE_DIMS[2], n_labels=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / N_EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
# train the network
print("[INFO] training network...")
H = model.fit(
    x=aug.flow(trainX, trainY, batch_size=N_BATCHES),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // N_BATCHES,
    epochs=N_EPOCHS, verbose=1)

[INFO] compiling model...
[INFO] training network...
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
16/16 [==============================] - 20s 1s/step - loss: 2.7275 - acc: 0.3957 - val_loss: 3.3945 - val_acc: 0.2126
Epoch 2/100
16/16 [==============================] - 17s 1s/step - loss: 1.5045 - acc: 0.5433 - val_loss: 2.5885 - val_acc: 0.1417
Epoch 3/100
16/16 [==============================] - 17s 1s/step - loss: 1.2883 - acc: 0.5787 - val_loss: 1.7900 - val_acc: 0.2205
Epoch 4/100
16/16 [==============================] - 17s 1s/step - loss: 1.0943 - acc: 0.6614 - val_loss: 1.6700 - val_acc: 0.3780
Epoch 5/100
16/16 [==============================] - 18s 1s/step - loss: 0.9412 - acc: 0.7126 - val_loss: 1.6054 - val_acc: 0.3071
Epoch 6/100
16/16 [==============================] - 19s 1s/step - loss: 0.8065 - acc: 0.7480 - val_loss: 1.7494 - val_acc: 0.2835
Epoch 7/100
16/16 [==============================] - 20s 1s/step - loss: 0.7783 - acc: 0.7323 - val_loss: 1.6179 

16/16 [==============================] - 19s 1s/step - loss: 0.4532 - acc: 0.8445 - val_loss: 0.7605 - val_acc: 0.7874
Epoch 38/100
16/16 [==============================] - 21s 1s/step - loss: 0.3884 - acc: 0.8583 - val_loss: 0.6905 - val_acc: 0.8189
Epoch 39/100
16/16 [==============================] - 18s 1s/step - loss: 0.4203 - acc: 0.8465 - val_loss: 0.6336 - val_acc: 0.7874
Epoch 40/100
16/16 [==============================] - 19s 1s/step - loss: 0.3093 - acc: 0.8917 - val_loss: 0.5554 - val_acc: 0.8346
Epoch 41/100
16/16 [==============================] - 24s 2s/step - loss: 0.3553 - acc: 0.8701 - val_loss: 0.6365 - val_acc: 0.7795
Epoch 42/100
16/16 [==============================] - 18s 1s/step - loss: 0.2948 - acc: 0.8937 - val_loss: 0.6676 - val_acc: 0.7559
Epoch 43/100
16/16 [==============================] - 17s 1s/step - loss: 0.3318 - acc: 0.8839 - val_loss: 1.2237 - val_acc: 0.6929
Epoch 44/100
16/16 [==============================] - 17s 1s/step - loss: 0.4342 - acc: 0

Epoch 75/100
16/16 [==============================] - 18s 1s/step - loss: 0.2897 - acc: 0.8839 - val_loss: 1.1877 - val_acc: 0.6929
Epoch 76/100
16/16 [==============================] - 17s 1s/step - loss: 0.2597 - acc: 0.8937 - val_loss: 1.3933 - val_acc: 0.7008
Epoch 77/100
16/16 [==============================] - 17s 1s/step - loss: 0.2619 - acc: 0.8976 - val_loss: 1.0464 - val_acc: 0.7480
Epoch 78/100
16/16 [==============================] - 17s 1s/step - loss: 0.2173 - acc: 0.9193 - val_loss: 0.6855 - val_acc: 0.8425
Epoch 79/100
16/16 [==============================] - 17s 1s/step - loss: 0.2735 - acc: 0.9055 - val_loss: 0.9004 - val_acc: 0.7638
Epoch 80/100
16/16 [==============================] - 17s 1s/step - loss: 0.2153 - acc: 0.9350 - val_loss: 1.0734 - val_acc: 0.6929
Epoch 81/100
16/16 [==============================] - 17s 1s/step - loss: 0.2210 - acc: 0.9055 - val_loss: 2.1005 - val_acc: 0.6142
Epoch 82/100
16/16 [==============================] - 17s 1s/step - loss: 0.

In [48]:
# save the model to disk
print("[INFO] serializing network...")
#model.save(args["model"], save_format="h5")
model.save(MODEL_PATH + 'model.h5')

# save the label binarizer to disk
print("[INFO] serializing label binarizer...")
#f = open(args["labelbin"], "wb")
f = open(MODEL_PATH +'label.bin', "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] serializing network...
[INFO] serializing label binarizer...


In [51]:
print(H.history.keys())

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])


In [55]:
plt.style.use("ggplot")
plt.figure()
N = N_EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig(PLOT_PATH)